# **DATA WRANGLING SITUACIÓN ESTACIONES**
This file attemps to gather, collect, and transform the station situations raw dataset from the source attached below in order to analyse the data avilable and proceed with it. Data collected dates July 2019 to December 2019 (pre-covid). The following processes will be dealt with:

1. Reading the .json files and transforming variables
2. Data exploration
3. Reshaping data
4. Filtering data

<u>Source</u>: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=f4b07e0543815610VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

<u>Folder</u>: Situación estaciones BiciMAD por día y hora entre julio y diciembre de 2019 

#### **LIBRARIES**

In [ ]:
import pandas as pd 
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots

#### **1. READ DATA and VARIABLE TRANSFORMATION**
**Dataset**: 20190x.json (X stands for the month number)    

**Description**: Dataset of the situation of the different stations of BiciMAD by date and time

**Dataframe size**: Each month 145,286 timestamps are colected with information on 14 variables. Observations froom july to december total 883,777 timestamps

**Variables**:
- _id_: Código de la Estación Base
- _latitude_: Latitud de la estación en formato WGS84
- _longitude_: Longitud de la Estación en formato WGS84
- _name_: Nombre de la Estación
- _light_: Grado de Ocupación (0=baja, 1=media, 2=alta, 3=inactiva)
- _number_: Denominación lógica de la Estación Base
- _actívate_: Estación activa (0=No activa, 1=activa)
- _no_available_: Disponibilidad de la Estación (0=disponible, 1=no disponible)
- _total_bases_: Número de bases de la estación
- _dock_bikes_: Número de bicicletas ancladas
- _free_bases_: Número de bases libres
- _reservations_count_: Número de reservas activas

In [ ]:
situaciones_list = ["201907", "201908", "201909", "201910", "201911", "201912"]

situaciones = pd.DataFrame()


for i in situaciones_list:
    data = []
    with open('../Data/SituacionEstaciones/'+i+'.json','r') as f:
        for line in f:
            data.append(json.loads(line))

    df_i = pd.json_normalize(
        data, 
        meta=['_id'],
        record_path =['stations']
    )
    # Convert _id to format date
    df_i["_id"] = pd.to_datetime(df_i["_id"])
    # Dorp column id of the base
    situaciones = pd.concat( [situaciones, df_i.drop(columns="id")], ignore_index=True, axis=0)

situaciones.head()

#### **2. DATA EXPLORATION**

In [ ]:
situaciones.shape

In [ ]:
situaciones.to_parquet('../Data/SituacionEstaciones/situaciones.parquet')

In [ ]:
situaciones.head(10)

**Variables type check**: correct

In [ ]:
situaciones.dtypes

**NaN check**: don't exist

In [ ]:
situaciones.isna().sum()

All stations are active. Variable **activate** is irrelevant and can be dropped

In [ ]:
situaciones["activate"].value_counts()

In [ ]:
situaciones.value_counts(["name", "number"])

Distribution of **number of reservations** varies across **stations and months**. 
- December has the lowest number of reservations followed by august. This may beacuase of holidays
- San Hermenegildo is the station that gets more reservations (total=and San Juan the Lowest

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x = situaciones["name"],
        y = situaciones.groupby(situaciones["name"])["reservations_count"].sum(),
        name = "Julio"
    )
)
fig.update_layout(title = "Número de reservas total de reservas Jul-Dic 2019 por Estación y Mes", xaxis={'categoryorder':'total descending'})
fig.show()

In [ ]:
situaciones.head()

In [ ]:
situaciones['number'].unique()

In [ ]:
bases2=pd.read_excel('../Data/Bases/bases_bicimad.xls')
bases2['Número'].unique()


In [ ]:
bases2['number']=bases2['Número'].apply(lambda x: x.r)